In [ ]:
from flask import Flask, redirect, render_template, request, sessions, flash, url_for
from werkzeug.utils import secure_filename
import io
import numpy as np
import cv2
from keras.models import Sequential
from keras.optimizers import Adam

app = Flask(__name__)
ALLOWED_EXTENSIONS = {'pdf', 'png', 'jpg', 'jpeg',}
UPLOAD_FOLDER = 'templates/upload_folder'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/')
def index():
    return render_template('homepage.html')

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/upload_picture_mobilenet', methods=['GET', 'POST'])
def upload_picture_mobilenet():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No image file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            file.save(secure_filename(file.filename))

            dim = (224,224,3)   

            mb = get_mobilenet(dim, model_weights="A:\weapon\weights.h5")
            prob,str = func(mb,file.filename)
            return render_template('display_photo.html', bounding_image = f'{file.filename}.jpg', lime_image = f'{file.filename}Lime.jpg',
                                   cat = str , prob = prob)
            
           

    return '''
    <!doctype html>
    <head>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
            color: #333;
        }
        .container {
            max-width: 800px;
            margin: 20px auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        h1 {
            text-align: center;
            color: #0066ff;
        }
        form {
            text-align: center;
            margin-top: 20px;
        }
        input {
            background-color: #0066ff;
            color: #fff;
            padding: 10px 20px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s ease;
            margin-right: 10px;
        }
        input[type="submit"]:hover {
            background-color: #0052cc;
        }
    </style>
    </head>
    </body>
    <div class="container"><br/><br/><br/><br/>
    <h1>Mobilenet: Upload Any Image</h1><br/>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file><br/><br/><br/><br/>
      <input type=submit value=Predict>
    </form><br/><br/>
    <p><a href = '/'>Homepage</a><br/><br/><br/><br/>
    </div>
    <body>
    '''

if __name__ == '__main__':
    app.run(threaded = True, port = 5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Apr/2024 12:15:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Apr/2024 12:15:32] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing import image 
from keras.preprocessing.image import load_img 
import pandas as pd 
from keras.utils import to_categorical
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, f1_score, confusion_matrix,precision_score,recall_score,average_precision_score
from itertools import cycle
import itertools

In [2]:
def get_mobilenet(dim, model_weights="A:\weapon\weights.h5"):
    model = Sequential()
    optimizer = Adam(lr = .0005)
    baseModel = MobileNetV2(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(7, 7)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation="softmax", name = 'Output'))

    
    for layer in baseModel.layers:
        layer.trainable = False
        
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    model.load_weights('A:\weapon\weights.h5')
    return model

In [3]:
dim=(224,224)
DIM=(224,224)

In [4]:
def func(mb,filename):
    new_path=f'A://weapon//pictures//{filename}'
    a11 = get_img_array([new_path], dim)
    pred = mb.predict(a11)
    val = np.argmax(pred)
    if(val == 0):
        print("no weapon")
        str ="no weapon"
    elif(val == 1):
        print("pistol")
        str="pistol"
    elif(val == 2):
        print("rifle")
        str="rifle"
    else:
        print("error")
        str="error"
    return val,str

In [5]:
def get_image_value(path, dim):
    img = image.load_img(path, target_size = dim)
    img = image.img_to_array(img)
    
    return img/255
def get_img_array(img_paths, dim): 
    final_array = []
    for path in img_paths:
        img = get_image_value(path, dim)
        final_array.append(img)
    final_array = np.array(final_array)
    
    return final_array